In [ ]:
import pandas as pd

# Load the EVDS data
evds_data = pd.read_excel('EVDS.xlsx')

# Display the first few rows of the data 
evds_data.head()

In [ ]:
# Rename the columns: Tarih to Date, TP METALIHR G71 to Export Total, TP KREHACBS A1 to Total Domestic loan volume and TP HKFE01 to Housing Price Index
evds_data.rename(columns={'Tarih': 'Date', 'TP METALIHR G71': 'Export Total', 'TP KREHACBS A1': 'Total Domestic loan volume', 'TP DK USD A YTL': 'Exchange Rates'}, inplace=True)

evds_data.head()

In [ ]:
# Convert the date column to datetime, and the remaining columns to numeric
evds_data['Date'] = pd.to_datetime(evds_data['Date'], errors='coerce')

# Convert the remaining columns to numeric, coercing errors to NaN
evds_data['Export Total'] = pd.to_numeric(evds_data['Export Total'], errors='coerce')
evds_data['Total Domestic loan volume'] = pd.to_numeric(evds_data['Total Domestic loan volume'], errors='coerce')
evds_data['Exchange Rates'] = pd.to_numeric(evds_data['Exchange Rates'], errors='coerce')

# Check the resulting data types
evds_data.info()

In [206]:
# A function to automate converting the date column to datetime and Search Volume to integer

def preprocess_data(df, date_column='Week', search_volume_column='Custom Search Volume', search_volume_new_name='Custom Search Volume'):
    """
    Preprocesses the data by renaming the specified date column to 'Date', 
    converting it to datetime, and converting the specified search volume 
    column to numeric with a new specified name.

    Parameters:
    - df: pandas DataFrame to preprocess.
    - date_column: Name of the column to rename to 'Date' and convert to datetime. Default is 'Week'.
    - search_volume_column: Name of the column to convert to numeric. Default is 'Search Volume'.
    - search_volume_new_name: New name for the search volume column after conversion. Default is 'Custom Search Volume'.

    Returns:
    - The preprocessed pandas DataFrame.
    """
    # Check if the date column exists in the DataFrame
    if date_column in df.columns:
        # Rename the date column to 'Date' and convert it to datetime
        df[date_column] = pd.to_datetime(df[date_column])
        df.rename(columns={date_column: 'Date'}, inplace=True)
    else:
        print(f"Column '{date_column}' not found in the DataFrame.")

    # Check if the search volume column exists in the DataFrame
    if search_volume_column in df.columns:
        # Convert the search volume column to numeric, coercing errors
        df[search_volume_column] = pd.to_numeric(df[search_volume_column], errors='coerce')
        # Rename the search volume column to the new specified name
        df.rename(columns={search_volume_column: search_volume_new_name}, inplace=True)
    else:
        print(f"Column '{search_volume_column}' not found in the DataFrame.")

    return df

In [ ]:
# Load and inspect the Google Trends data for "exports"
exports_data = pd.read_csv('exports.csv')

# Display the first few rows of the data 
exports_data.head()

In [ ]:
# Processing the exports data
exports_data = preprocess_data(exports_data, date_column='Week', search_volume_column='Search Volume', search_volume_new_name='Exports Search Volume')

exports_data.head()

In [ ]:
# Load and inspect the Google Trends data for "USD-Lira"
usd_lira_data = pd.read_csv('USD-Lira.csv')

# Display the first few rows of the data 
usd_lira_data.head()

In [ ]:
# Processing the house price data
house_price_data = preprocess_data(usd_lira_data, date_column='Week', search_volume_column='Search Volume', search_volume_new_name='USD-Lira Search Volume')

house_price_data.head()

In [ ]:
# Load and inspect the Google Trends data for "loan"
loan_data = pd.read_csv('loan.csv')

# Inspection of the first few rows of the data
loan_data.head()

In [ ]:
# Processing the loan data
loan_data = preprocess_data(loan_data, date_column='Week', search_volume_column='Search Volume', search_volume_new_name='Loan Search Volume')

loan_data.head()

In [ ]:
# Load the interest rate data
interest_rate_data = pd.read_csv('interest rates.csv')

interest_rate_data.head()

In [ ]:
interest_rate_data = preprocess_data(interest_rate_data, date_column='Week', search_volume_column='Search Volume', search_volume_new_name='Interest Rate Search Volume')

interest_rate_data.head()

In [215]:
# Setting the 'Date' column as the index for each DataFrame
def set_date_as_index(df):
    """
    Sets the 'Date' column as the index of the DataFrame.

    Parameters:
    - df: pandas DataFrame to set the 'Date' column as index.

    Returns:
    - The DataFrame with the 'Date' column as index.
    """
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    else:
        print("Column 'Date' not found in the DataFrame.")

    return df

# Set the 'Date' column as the index for each DataFrame
evds_data = set_date_as_index(evds_data)
exports_data = set_date_as_index(exports_data)
usd_lira_data = set_date_as_index(usd_lira_data)
loan_data = set_date_as_index(loan_data)
interest_rate_data = set_date_as_index(interest_rate_data)

In [ ]:
exports_data.head(), loan_data.head(), interest_rate_data.head(), usd_lira_data.head()

In [217]:
# A function to aggregate the weekly data to monthly averages
def aggregate_weekly_data_to_monthly(df):
    """
    Aggregates the weekly data to monthly averages.

    Parameters:
    - df: pandas DataFrame with weekly data to aggregate.

    Returns:
    - The DataFrame with weekly data aggregated to monthly averages.
    """
    # Resample the weekly data to monthly frequency, taking the mean of each month
    df_monthly = df.resample('M').mean()

    return df_monthly

exports_data = aggregate_weekly_data_to_monthly(exports_data)
usd_lira_data = aggregate_weekly_data_to_monthly(usd_lira_data)
loan_data = aggregate_weekly_data_to_monthly(loan_data)
interest_rate_data = aggregate_weekly_data_to_monthly(interest_rate_data)

In [ ]:
exports_data.head(), usd_lira_data.head(), loan_data.head(), interest_rate_data.head(), evds_data.head()

In [ ]:
evds_data.index = pd.to_datetime(evds_data.index).to_period('M').to_timestamp('M')

evds_data.head()

In [ ]:
import matplotlib.pyplot as plt

# Plotting the monthly averages of Google Trends data
fig, axs = plt.subplots(4, 1, figsize=(14, 18))

# Exports
axs[0].plot(exports_data.index, exports_data['Exports Search Volume'], marker='o', linestyle='-', color='blue')
axs[0].set_title('Monthly Average Search Volume for Exports')
axs[0].set_ylabel('Search Volume')

# USD-Lira
axs[1].plot(usd_lira_data.index, usd_lira_data['USD-Lira Search Volume'], marker='o', linestyle='-', color='green')
axs[1].set_title('Monthly Average Search Volume for USD-Lira')
axs[1].set_ylabel('Search Volume')

# Loan
axs[2].plot(loan_data.index, loan_data['Loan Search Volume'], marker='o', linestyle='-', color='red')  # Typo in column name corrected here visually
axs[2].set_title('Monthly Average Search Volume for Loan')
axs[2].set_ylabel('Search Volume')

# Interest Rates
axs[3].plot(interest_rate_data.index, interest_rate_data['Interest Rate Search Volume'], marker='o', linestyle='-', color='purple')
axs[3].set_title('Monthly Average Search Volume for Interest Rates')
axs[3].set_ylabel('Search Volume')

plt.tight_layout()
plt.show()

In [ ]:
# To proceed with correlation analysis, we need to merge the EVDS data with the Google Trends aggregated monthly data

# Merge the datasets
merged_data = pd.concat([exports_data, usd_lira_data, loan_data, interest_rate_data, evds_data], axis=1)

merged_data.head()

In [ ]:
merged_data.tail()

In [ ]:
# Missing values in the merged data

merged_data.isnull().sum()

In [ ]:
# Remove the rows with missing values

merged_data = merged_data.dropna()

merged_data.head()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = merged_data.corr()

correlation_matrix

In [ ]:
# Plot the correlation matrix

import seaborn as sns

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt = ".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(merged_data.index, merged_data['Export Total'], label='Export Total')
plt.plot(merged_data.index, merged_data['Total Domestic loan volume'], label='Total Domestic Loan Volume')
plt.plot(merged_data.index, merged_data['Exchange Rates'], label='Exchange Rates')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Time Series Plot of Economic Indicators')
plt.legend()
plt.show()

In [228]:
Y = merged_data['Exchange Rates']
X = merged_data[['Exports Search Volume', 'USD-Lira Search Volume', 'Loan Search Volume', 'Interest Rate Search Volume']]

split_point = int(len(X) * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
Y_train, Y_test = Y[:split_point], Y[split_point:]

In [ ]:
# Building a ARIMA Model 

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

exchange_rates = merged_data['Exchange Rates']

# Plot ACF and PACF
plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(exchange_rates, ax=plt.gca(), lags=10)
plt.subplot(122)
plot_pacf(exchange_rates, ax=plt.gca(), lags=10)
plt.show()

In [ ]:
# Grid search to get ARIMA values
import itertools
import warnings
warnings.filterwarnings("ignore")

# Define the p, d, and q ranges to test
p = range(0, 3)  
d = range(0, 2)  
q = range(0, 3) 
pdq = list(itertools.product(p, d, q))

# Grid search
best_aic = float('inf')
best_order = None
best_model = None

for param in pdq:
    try:
        temp_model = ARIMA(exchange_rates, order=param)
        results = temp_model.fit()
        
        # Compare current model's AIC with the best so far
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = param
            best_model = results
    except:  # Handle cases where the model fails to converge
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')
print(best_model.summary())

In [ ]:
best_model.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Log transformation Exports total to stabilize variance
import numpy as np

merged_data['Log Export Total'] = np.log(merged_data['Export Total'])

merged_data[['Export Total', 'Log Export Total']].head()

In [242]:
# Define the dependent and independent variables
y_export = merged_data['Log Export Total']
X_export = merged_data[['Exports Search Volume', 'USD-Lira Search Volume', 'Loan Search Volume', 'Interest Rate Search Volume']]

split_point = int(len(X) * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
Y_train, Y_test = Y[:split_point], Y[split_point:]

In [ ]:
# Building a ARIMA Model due to presence of autocorrelation and potential issues with normality of residuals

exports_total = merged_data['Log Export Total']

# Plot ACF and PACF
plt.figure(figsize=(12, 6))

plt.subplot(121)
plot_acf(exports_total, ax=plt.gca(), lags=10)
plt.subplot(122)
plot_pacf(exports_total, ax=plt.gca(), lags=10)
plt.show()

In [ ]:
# Grid search to get ARIMA values

# Define the p, d, and q ranges to test
p = range(0, 3)  
d = range(0, 2)  
q = range(0, 3) 
pdq = list(itertools.product(p, d, q))

# Grid search
best_aic = float('inf')
best_order = None
best_model = None

for param in pdq:
    try:
        temp_model = ARIMA(exports_total, order=param)
        results = temp_model.fit()
        
        # Compare current model's AIC with the best so far
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = param
            best_model = results
    except:  # Handle cases where the model fails to converge
        continue
    
print(f"Best ARIMA{best_order} AIC: {best_aic}")
print(best_model.summary())

In [ ]:
best_model.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Log transformation of the dependent variable
merged_data['Log Domestic Loan Volume'] = np.log(merged_data['Total Domestic loan volume'])

merged_data[['Total Domestic loan volume', 'Log Domestic Loan Volume']].head()

In [252]:
# Define the dependent and independent variables
y_total_loan = merged_data['Log Domestic Loan Volume']
X_total_loan = merged_data[['Exports Search Volume', 'USD-Lira Search Volume', 'Loan Search Volume', 'Interest Rate Search Volume']]

split_point = int(len(X) * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
Y_train, Y_test = Y[:split_point], Y[split_point:]

In [ ]:
# ARIMA Model for Total Domestic Loan Volume

total_loan_volume = merged_data['Log Domestic Loan Volume']

# Plot ACF and PACF
plt.figure(figsize=(12, 6))

plt.subplot(121)
plot_acf(total_loan_volume, ax=plt.gca(), lags=10)
plt.subplot(122)
plot_pacf(total_loan_volume, ax=plt.gca(), lags=10)
plt.show()

In [ ]:
# Grid search to get ARIMA values

# Define the p, d, and q ranges to test
p = range(0, 3)
d = range(0, 2)
q = range(0, 3)
pdq = list(itertools.product(p, d, q))

# Grid search
best_aic = float('inf')
best_order = None
best_model = None

for param in pdq:
    try:
        temp_model = ARIMA(total_loan_volume, order=param)
        results = temp_model.fit()
        
        # Compare current model's AIC with the best so far
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = param
            best_model = results
    except:  # Handle cases where the model fails to converge
        continue
    
print(f"Best ARIMA{best_order} AIC: {best_aic}")
print(best_model.summary())

In [ ]:
best_model.plot_diagnostics(figsize=(15, 12))
plt.show()